In [3]:
import csv
import random
import re
import numpy as np
import pandas as pd
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.svm import SVC
from sklearn.metrics import f1_score
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.utils import np_utils
import nltk
from nltk.tokenize import TweetTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords

data = pd.read_csv("/content/drive/MyDrive/Temp/Constraint_English_Train - Sheet1.csv")
data = data.drop('id', axis=1)
#print(data)
train_X = data.values[:,0]
train_Y = data.values[:,1]
#print(train_X)
#print(train_Y)
dataval = pd.read_csv("/content/drive/MyDrive/Temp/Constraint_English_Val - Sheet1.csv")
dataval = dataval.drop('id', axis=1)
#print(dataval)
val_X = dataval.values[:,0]
val_Y = dataval.values[:,1]

stopwords_set = set(stopwords.words("english"))

Xtrain = []
for s in train_X:
  #print(s)
  r = re.sub(r"http\S+|@\S+", "", s)
  r_lower =r.lower()
  text = TweetTokenizer().tokenize(r_lower)
  sent = ' '.join([w for w in text if w not in stopwords_set])
  Xtrain.append(sent)

Xval = []
for s in val_X:
  #print(s)
  r = re.sub(r"http\S+|@\S+", "", s)
  r_lower =r.lower()
  text = TweetTokenizer().tokenize(r_lower)
  sent = ' '.join([w for w in text if w not in stopwords_set])
  Xval.append(sent)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
print(Xtrain)
print(Xval)

['cdc currently reports 99031 deaths . general discrepancies death counts different sources small explicable . death toll stands roughly 100000 people today .', 'states reported 1121 deaths small rise last tuesday . southern states reported 640 deaths .', 'politically correct woman ( almost ) uses pandemic excuse reuse plastic bag #coronavirus #nashville', '#indiafightscorona : 1524 #covid testing laboratories india 25th august 2020 3682752 0 tests done : dg #staysafe #indiawillwin', 'populous states generate large case counts look new cases per million today 9 smaller states showing cases per million california texas : al ar id ks ky la ms nv sc .', 'covid act found " average person illinois covid - 19 infecting 1.11 people . data shows infection growth rate declined time factors stay-at-home order restrictions put place . "', 'tested positive #covid19 symptoms stay home away people . learn cdc ’ recommendations around others covid - 19 infection :', 'obama calls trump ’ coronavirus r

In [14]:
def split_train_val(X_train, op_train):
  samples = list(zip(X_train,op_train))  #combine the representation
  random.seed(13)
  random.shuffle(samples) #random shuffling
  length=len(samples)
  k=int(0.8*length)
  X_train, op_train = zip(*samples[80:220])
  X_test, op_test= zip(*samples[:80])
  return np.array(X_train),np.array(op_train),np.array(X_test),np.array(op_test)
def build_vocab(docs):
    vocab = []
    for doc in docs:
        vocab += doc.split()
    vocab = list(set(vocab))
    w_to_i = {w:i for i,w in enumerate(vocab,1)} #enumerate from 1 to avoid confusion with zero padding
    i_to_w = {i:w for i,w in enumerate(vocab,1)}

    return w_to_i, i_to_w
def one_hot(docs,w_to_i):
    embeddings = []
    for doc in docs:
        embeddings.append([w_to_i[word] for word in doc.split() if word in w_to_i])

    return embeddings
def shuffle_list(*ls):
    l =list(zip(*ls))
    random.shuffle(l)
    return zip(*l)
def predict_operator(X_train, y_train, X_test, y_test,TEST):
    one_hot_dict, inv_one_hot_dict = build_vocab(X_train)
    vocabulary_size = len(one_hot_dict)+1
    X_train_embedding = one_hot(X_train,one_hot_dict)
    X_test_embedding = one_hot(X_test,one_hot_dict)
    TEST_embedding = one_hot(TEST,one_hot_dict)
    category_num = len(set(y_train))
    categories = list(set(y_train))
    o_to_i = {o:i for i,o in enumerate(categories)}
    i_to_o = {i:o for i,o in enumerate(categories)}
    y_train_numeric = []
    y_test_numeric = []
    for label in y_train:
        y_train_numeric.append(o_to_i[label])
    for label in y_test:
        y_test_numeric.append(o_to_i[label])

    y_train_distribution = np_utils.to_categorical(y_train_numeric, category_num)
    y_test_distribution = np_utils.to_categorical(y_test_numeric, category_num)

    # pad documents to max length
    max_length = 100
    X_train_embedding_padded = pad_sequences(X_train_embedding, maxlen=max_length, padding='post')
    X_test_embedding_padded = pad_sequences(X_test_embedding, maxlen=max_length, padding='post')
    TEST_embedding_padded = pad_sequences(TEST_embedding, maxlen=max_length, padding='post')


    X_shuffled, y_shuffled = shuffle_list(X_train_embedding_padded, y_train_distribution)
    length = len(X_shuffled)
    X_train_onehot = np.array(X_shuffled[:int(0.8*length)])
    X_dev_onehot = np.array(X_shuffled[int(0.8*length):])
    #X_TEST_onehot = np.array(X_shuffled[int(0.8*length):])
    y_train_distribution = np.array(y_shuffled[:int(0.8*length)])
    y_dev_distribution = np.array(y_shuffled[int(0.8*length):])
    '''
    embedding_dim = 256
    filter_sizes = [3, 4, 5]
    num_filters = 70
    drop = 0.2

    epochs = 70
    batch_size = 128
    '''
    embedding_dim = 256
    filter_sizes = [3, 4, 5, 6, 7]
    num_filters = 64
    drop = 0.2

    epochs = 10
    batch_size = 128

    print("Creating Model...")
    inputs = Input(shape=(max_length,), dtype='int32')
    embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=max_length)(inputs)
    reshape = Reshape((max_length, embedding_dim, 1))(embedding)

    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
    conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
    conv_4 = Conv2D(num_filters, kernel_size=(filter_sizes[4], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
    

    maxpool_0 = MaxPool2D(pool_size=(max_length - filter_sizes[0] + 1, 1), strides=(1, 1), padding='valid')(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(max_length - filter_sizes[1] + 1, 1), strides=(1, 1), padding='valid')(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(max_length - filter_sizes[2] + 1, 1), strides=(1, 1), padding='valid')(conv_2)
    maxpool_3 = MaxPool2D(pool_size=(max_length - filter_sizes[3] + 1, 1), strides=(1, 1), padding='valid')(conv_3)
    maxpool_4 = MaxPool2D(pool_size=(max_length - filter_sizes[4] + 1, 1), strides=(1, 1), padding='valid')(conv_4)
    #maxpool_5 = MaxPool2D(pool_size=(max_length - filter_sizes[5] + 1, 1), strides=(1, 1), padding='valid')(conv_5)

    concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3, maxpool_4])
    flatten = Flatten()(concatenated_tensor)
    dropout = Dropout(drop)(flatten)
    output = Dense(units=category_num, activation='softmax')(dropout)

    model = Model(inputs=inputs, outputs=output)

    checkpoint = ModelCheckpoint('CNN.hdf5', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
    adam = Adam(lr=2e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    #print(model.summary())
    print("Training Model...")
    model.fit(X_train_onehot, y_train_distribution, batch_size=batch_size, epochs=epochs, verbose=0, callbacks=[checkpoint],
         validation_data=(X_dev_onehot, y_dev_distribution))

    model.load_weights("CNN.hdf5")
    y_predict = model.predict(X_test_embedding_padded)
    y_predict = [i_to_o[np.argmax(i)] for i in y_predict]
    yscore=[]
    yreal=[]
    for i1 in range(len(y_test)):
      if y_test[i1]=='fake':
        yreal.append(1)
      else:
        yreal.append(2)
      if y_predict[i1]=='fake':
        yscore.append(1)
      else:
        yscore.append(2)
    print("roc auc score: ",roc_auc_score(yreal, yscore, average=None))
    TEST_preds = model.predict(TEST_embedding_padded)
    TEST_preds = [i_to_o[np.argmax(i)] for i in TEST_preds]
    #print(y_predict)
    print("Operation acc: {}".format(model.evaluate(X_test_embedding_padded,y_test_distribution,verbose=0)[1]))
    #return y_predict
    return TEST_preds

In [15]:
from sklearn.metrics import roc_auc_score
datatest = pd.read_csv("/content/drive/MyDrive/Temp/Constraint_English_Test - Sheet1.csv")
#datatest = data.drop('id', axis=1)
#print(data)
test_X = datatest.values[:,1]
#print(test_X)
cnn = predict_operator(Xtrain, train_Y, Xval, val_Y, test_X)
print(cnn)
#print("Weighted f1: ",f1_score(val_Y, cnn, average='weighted'))

Creating Model...
Training Model...
roc auc score:  0.9496411064425769
Operation acc: 0.949532687664032
['real', 'fake', 'fake', 'real', 'real', 'fake', 'real', 'real', 'real', 'real', 'real', 'fake', 'real', 'real', 'fake', 'fake', 'fake', 'fake', 'fake', 'real', 'fake', 'fake', 'real', 'real', 'real', 'real', 'real', 'real', 'real', 'fake', 'fake', 'fake', 'real', 'fake', 'real', 'fake', 'real', 'real', 'fake', 'real', 'fake', 'fake', 'fake', 'fake', 'fake', 'real', 'real', 'fake', 'fake', 'real', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'real', 'fake', 'fake', 'real', 'fake', 'real', 'fake', 'fake', 'fake', 'fake', 'real', 'fake', 'real', 'real', 'real', 'fake', 'fake', 'real', 'real', 'real', 'fake', 'real', 'fake', 'fake', 'real', 'fake', 'real', 'real', 'real', 'fake', 'real', 'real', 'real', 'fake', 'real', 'real', 'fake', 'real', 'fake', 'real', 'fake', 'real', 'real', 'fake', 'real', 'fake', 'fake', 'fake', 'fake', 'fake', 'real', 'real', 'real', 'real', 'real', 'fake',

In [ ]:
opf = open("answer.txt","w")
opf.write("id,label\n")
ids = datatest.values[:,0]
for i in range(len(cnn)):
  opf.write(str(ids[i])+","+cnn[i]+"\n")
  print(str(ids[i])+","+cnn[i]+"\n")
opf.close()

1,real

2,fake

3,fake

4,real

5,real

6,fake

7,real

8,real

9,real

10,real

11,real

12,fake

13,real

14,real

15,fake

16,fake

17,real

18,fake

19,fake

20,real

21,fake

22,fake

23,fake

24,fake

25,real

26,real

27,real

28,fake

29,real

30,fake

31,fake

32,fake

33,real

34,fake

35,real

36,real

37,real

38,real

39,fake

40,real

41,real

42,fake

43,fake

44,fake

45,fake

46,real

47,real

48,real

49,fake

50,real

51,fake

52,fake

53,fake

54,fake

55,fake

56,fake

57,real

58,fake

59,fake

60,real

61,fake

62,real

63,fake

64,fake

65,fake

66,fake

67,real

68,fake

69,real

70,real

71,real

72,fake

73,fake

74,real

75,real

76,real

77,fake

78,real

79,fake

80,fake

81,real

82,fake

83,real

84,real

85,real

86,fake

87,real

88,real

89,real

90,fake

91,real

92,fake

93,fake

94,real

95,fake

96,fake

97,fake

98,real

99,real

100,real

101,real

102,fake

103,real

104,fake

105,fake

106,fake

107,real

108,real

109,fake

110,real

111,real